Open Play Shots - Expected Score Model (Multiclass) - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 1
model_name = 'expected_score_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [4]:
RESPONSE = ModellingDataContract.RESPONSE_MULTICLASS

Load Data

In [5]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,16,True
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,43,True
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,5,True
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,88,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,23,True


In [6]:
df_modelling['Kick_Outcome'] = df_modelling['Final_State'].map({'miss':0, 'behind':1, 'goal':2})

In [7]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [8]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [9]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
2,10,possGain,goal,110.0,1,285,306.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,50.0,24.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,0,6,6,0,12.0,0.0,12.0,Kick,Gather,Kick Inside 50 Result,Kick Into F50,50.0,50.0,50.0,-8.0,24.0,28.0,28.0,44.0,306.0,305.0,304.0,303.0,21.0,1.0,4.000000,28.0,24.0,36.878178,0.708626,40.601295,0.132764,7.606837,0.424028,24.295030,784.0,3.332205,False,83,False
7,50,possGain,behind,512.0,1,1462,1500.0,Brisbane Lions,Brisbane Lions,Oscar McInerney,Oscar_McInerney,Kick,65.0,-15.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,0,1,1,0,19.0,15.0,4.0,Kick,Hard Ball Get Crumb,Spoil,Kick Inside 50 Result,65.0,62.0,60.0,60.0,-15.0,-16.0,-17.0,-17.0,1500.0,1499.0,1497.0,1496.0,38.0,1.0,3.162278,13.0,15.0,19.849433,0.856706,49.085617,0.216802,12.421849,0.826928,47.379494,169.0,2.564949,False,90,False
8,79,possGain,goal,747.0,2,709,720.0,Sydney,Sydney,Sam Wicks,Sam_Wicks,Kick,41.0,18.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,0,6,0,6,20.0,40.0,-20.0,Kick,Handball Received,Handball,Handball Received,41.0,38.0,37.0,36.0,18.0,16.0,15.0,15.0,720.0,720.0,719.0,718.0,11.0,0.0,3.605551,37.0,18.0,41.146081,0.452778,25.942295,0.140721,8.062731,0.443767,25.425977,1369.0,3.610918,False,83,False
14,114,possGain,behind,1011.0,2,2025,2032.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,54.0,35.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,0,1,1,0,50.0,52.0,-2.0,Kick,Loose Ball Get,Kick Inside 50 Result,Kick Into F50,54.0,49.0,41.0,-9.0,35.0,33.0,31.0,42.0,2032.0,2029.0,2027.0,2027.0,7.0,3.0,5.385165,24.0,35.0,42.438190,0.969723,55.561011,0.085774,4.914467,0.269657,15.450186,576.0,3.178054,False,87,False
15,123,centreBounce,goal,1086.0,3,128,143.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,32.0,8.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,0,6,6,0,51.0,58.0,-7.0,Kick,Handball Received,Handball,Gather,32.0,22.0,26.0,26.0,8.0,18.0,20.0,20.0,143.0,140.0,139.0,139.0,15.0,3.0,14.142136,46.0,8.0,46.690470,0.172191,9.865807,0.135683,7.774075,0.423021,24.237313,2116.0,3.828641,False,92,False


Preprocess Data

In [10]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [11]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [12]:
X_train_preproc.shape, X_test_preproc.shape

((7208, 2), (2008, 2))

In [13]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
3,46.486557,0.328553
4,39.560081,0.281772
5,59.682493,0.546167


In [14]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
2,36.878178,0.708626
7,19.849433,0.856706
8,41.146081,0.452778
14,42.438190,0.969723
15,46.690470,0.172191


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [15]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, num_class=3)

In [16]:
xgb_tuner.training_data.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
3,46.486557,0.328553
4,39.560081,0.281772
5,59.682493,0.546167


In [17]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 16:28:33,313] A new study created in memory with name: no-name-572a4a0b-80ba-456b-b4cc-9b5903b2fe6d
[I 2023-06-28 16:28:33,396] Trial 0 finished with value: 1.0869075391721654 and parameters: {'max_depth': 3, 'min_child_weight': 16, 'eta': 0.011998976731631075, 'gamma': 0.08567835782748191, 'lambda': 0.0017350493816235774, 'alpha': 0.28322060448748293, 'subsample': 0.8412699865711892, 'colsample_bytree': 0.8504846895397047}. Best is trial 0 with value: 1.0869075391721654.
[I 2023-06-28 16:28:33,450] Trial 1 finished with value: 1.063829379818267 and parameters: {'max_depth': 11, 'min_child_weight': 79, 'eta': 0.05245090637964539, 'gamma': 0.7806452265891803, 'lambda': 0.1810286064534875, 'alpha': 5.4360757956384065, 'subsample': 0.6027646926198726, 'colsample_bytree': 0.7190537109162038}. Best is trial 1 with value: 1.063829379818267.
[I 2023-06-28 16:28:33,530] Trial 2 finished with value: 1.0410217503214536 and parameters: {'max_depth': 15, 'min_child_weight': 12, 'eta'

Number of finished trials:  100
Best trial:
  Value: 0.997639484921331
  Params: 
    max_depth: 10
    min_child_weight: 70
    eta: 0.47932972728727347
    gamma: 0.12948586948956814
    lambda: 0.0006061951182999966
    alpha: 0.0003477234025231466
    subsample: 0.30009869457851274
    colsample_bytree: 0.2865929647179113


In [18]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 10,
 'min_child_weight': 70,
 'eta': 0.47932972728727347,
 'gamma': 0.12948586948956814,
 'lambda': 0.0006061951182999966,
 'alpha': 0.0003477234025231466,
 'subsample': 0.30009869457851274,
 'colsample_bytree': 0.2865929647179113}

Training Model - SuperXGBClassifier class for training and predictions

In [31]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = {}
params['num_class'] = 3

In [32]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [33]:
super_xgb.fit()

[0]	validation_0-mlogloss:1.06050	validation_1-mlogloss:1.06053
[1]	validation_0-mlogloss:1.03436	validation_1-mlogloss:1.03640
[2]	validation_0-mlogloss:1.01765	validation_1-mlogloss:1.02277
[3]	validation_0-mlogloss:1.01061	validation_1-mlogloss:1.01733
[4]	validation_0-mlogloss:1.00364	validation_1-mlogloss:1.01074
[5]	validation_0-mlogloss:1.00091	validation_1-mlogloss:1.00924
[6]	validation_0-mlogloss:0.99910	validation_1-mlogloss:1.00866
[7]	validation_0-mlogloss:0.99692	validation_1-mlogloss:1.00820
[8]	validation_0-mlogloss:0.99548	validation_1-mlogloss:1.00695
[9]	validation_0-mlogloss:0.99387	validation_1-mlogloss:1.00738
[10]	validation_0-mlogloss:0.99268	validation_1-mlogloss:1.00974
[11]	validation_0-mlogloss:0.99186	validation_1-mlogloss:1.00829
[12]	validation_0-mlogloss:0.99123	validation_1-mlogloss:1.00970
[13]	validation_0-mlogloss:0.99006	validation_1-mlogloss:1.00737
[14]	validation_0-mlogloss:0.98953	validation_1-mlogloss:1.00717
[15]	validation_0-mlogloss:0.98875	

In [34]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.2865929647179113, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.12948586948956814, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.47932972728727347, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=70, missing=nan, monotone_constraints={},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [35]:
super_xgb.xgb_model.get_booster().feature_names

['Distance_to_Middle_Goal', 'Angle_to_Middle_Goal']

In [36]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [41]:
train_miss_probas = super_xgb.predict_proba(X_train_preproc)[:, 0]
train_behind_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
train_goal_probas = super_xgb.predict_proba(X_train_preproc)[:, 2]

test_miss_probas = super_xgb.predict_proba(X_test_preproc)[:, 0]
test_behind_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
test_goal_probas = super_xgb.predict_proba(X_test_preproc)[:, 2]

Check Average Predictions

In [42]:
train_miss_probas.mean(), training_data[ModellingDataContract.RESPONSE_MISS].mean(), test_miss_probas.mean(), test_data[ModellingDataContract.RESPONSE_MISS].mean()

(0.20773862, 0.2081021087680355, 0.20744663, 0.20418326693227093)

In [43]:
train_behind_probas.mean(), training_data[ModellingDataContract.RESPONSE_BEHIND].mean(), test_behind_probas.mean(), test_data[ModellingDataContract.RESPONSE_BEHIND].mean()

(0.3605883, 0.36764705882352944, 0.35986784, 0.3545816733067729)

In [44]:
train_goal_probas.mean(), training_data[ModellingDataContract.RESPONSE_GOAL].mean(), test_goal_probas.mean(), test_data[ModellingDataContract.RESPONSE_GOAL].mean()

(0.43167308, 0.4242508324084351, 0.43268552, 0.44123505976095617)

Export model

In [45]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [47]:
train_info = training_data[['Match_ID', "Chain_Number", RESPONSE, "Final_State"]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_miss_probas'] = train_miss_probas
train_export['xgb_behind_probas'] = train_behind_probas
train_export['xgb_goal_probas'] = train_goal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Kick_Outcome,Final_State,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_miss_probas,xgb_behind_probas,xgb_goal_probas
0,202101_BrisbaneLions_Sydney,1,2,goal,56.080300,0.383818,1,0.214769,0.450440,0.334791
1,202101_BrisbaneLions_Sydney,6,2,goal,47.010637,0.416065,1,0.274013,0.420308,0.305678
2,202101_BrisbaneLions_Sydney,21,2,goal,46.486557,0.328553,1,0.256617,0.461644,0.281738
3,202101_BrisbaneLions_Sydney,38,0,miss,39.560081,0.281772,1,0.255382,0.407632,0.336986
4,202101_BrisbaneLions_Sydney,43,1,behind,59.682493,0.546167,1,0.225220,0.428701,0.346079


In [48]:
test_info = test_data[['Match_ID', "Chain_Number", RESPONSE, "Final_State"]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_miss_probas'] = test_miss_probas
test_export['xgb_behind_probas'] = test_behind_probas
test_export['xgb_goal_probas'] = test_goal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Kick_Outcome,Final_State,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_miss_probas,xgb_behind_probas,xgb_goal_probas
0,202101_BrisbaneLions_Sydney,10,2,goal,36.878178,0.708626,1,0.229561,0.454029,0.316410
1,202101_BrisbaneLions_Sydney,50,1,behind,19.849433,0.856706,2,0.187500,0.287621,0.524879
2,202101_BrisbaneLions_Sydney,79,2,goal,41.146081,0.452778,2,0.235523,0.346838,0.417639
3,202101_BrisbaneLions_Sydney,114,1,behind,42.438190,0.969723,1,0.263734,0.425805,0.310462
4,202101_BrisbaneLions_Sydney,123,2,goal,46.690470,0.172191,1,0.253656,0.384936,0.361408


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")